# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf

### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [0]:
x_train_0to4 = x_train[y_train[:, 0] < 5]
x_test_0to4 = x_test[y_test[:, 0] < 5]
y_train_0to4 = y_train[y_train[:, 0] < 5]
y_test_0to4 = y_test[y_test[:, 0] < 5]

x_train_5to9 = x_train[y_train[:, 0] > 4]
x_test_5to9 = x_test[y_test[:, 0] > 4]
y_train_5to9 = y_train[y_train[:, 0] > 4]
y_test_5to9 = y_test[y_test[:, 0] > 4]

In [21]:
x_train_0to4.shape

(25000, 32, 32, 3)

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
num_classes = 10

y_train_0to4 = tf.keras.utils.to_categorical(y_train_0to4, num_classes)
y_test_0to4 = tf.keras.utils.to_categorical(y_test_0to4, num_classes)

y_train_5to9 = tf.keras.utils.to_categorical(y_train_5to9, num_classes)
y_test_5to9 = tf.keras.utils.to_categorical(y_test_5to9, num_classes)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Reshape((32,32,3),input_shape=(32,32,3)))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add first convolutional layer
model.add(tf.keras.layers.Conv2D(256, kernel_size=(3,3), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add second convolutional layer
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add second convolutional layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

#Add MaxPooling layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

In [0]:
#Flatten the output
model.add(tf.keras.layers.Flatten())

#Dense layer
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Add another dropout layer
model.add(tf.keras.layers.Dropout(0.25))

#Output layer
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 32, 32, 3)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 256)       7168      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 256)       1024      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 128)       295040    
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 128)       512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        7

In [51]:
#Train the model
model.fit(x_train_0to4,y_train_0to4,          
          validation_data=(x_test_0to4,y_test_0to4),
          epochs=30,
          batch_size=32)

Train on 25000 samples, validate on 5000 samples
Epoch 1/30
25000/25000 [==============================] - 9s 362us/sample - loss: 1.1749 - accuracy: 0.5685 - val_loss: 0.8234 - val_accuracy: 0.6666
Epoch 2/30
25000/25000 [==============================] - 8s 329us/sample - loss: 0.8145 - accuracy: 0.6840 - val_loss: 0.7310 - val_accuracy: 0.7146
Epoch 3/30
25000/25000 [==============================] - 8s 333us/sample - loss: 0.6965 - accuracy: 0.7377 - val_loss: 0.6845 - val_accuracy: 0.7484
Epoch 4/30
25000/25000 [==============================] - 8s 331us/sample - loss: 0.6112 - accuracy: 0.7681 - val_loss: 0.5973 - val_accuracy: 0.7760
Epoch 5/30
25000/25000 [==============================] - 8s 330us/sample - loss: 0.5453 - accuracy: 0.7991 - val_loss: 0.6267 - val_accuracy: 0.7730
Epoch 6/30
25000/25000 [==============================] - 8s 329us/sample - loss: 0.4692 - accuracy: 0.8246 - val_loss: 0.7864 - val_accuracy: 0.7390
Epoch 7/30
25000/25000 [===========================

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
for layer in model.layers[0:10]:
  layer.trainable = False

In [56]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 32, 32, 3)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 256)       7168      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 256)       1024      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 128)       295040    
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 128)       512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        7

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [57]:
model.fit(x_train_5to9,y_train_5to9,          
          validation_data=(x_test_5to9,y_test_5to9),
          epochs=30,
          batch_size=32)

Train on 25000 samples, validate on 5000 samples
Epoch 1/30
25000/25000 [==============================] - 5s 188us/sample - loss: 3.3892 - accuracy: 0.5300 - val_loss: 0.6091 - val_accuracy: 0.7820
Epoch 2/30
25000/25000 [==============================] - 4s 166us/sample - loss: 0.6350 - accuracy: 0.7577 - val_loss: 0.4847 - val_accuracy: 0.8326
Epoch 3/30
25000/25000 [==============================] - 4s 164us/sample - loss: 0.4539 - accuracy: 0.8312 - val_loss: 0.4366 - val_accuracy: 0.8490
Epoch 4/30
25000/25000 [==============================] - 4s 167us/sample - loss: 0.3554 - accuracy: 0.8700 - val_loss: 0.4278 - val_accuracy: 0.8554
Epoch 5/30
25000/25000 [==============================] - 4s 166us/sample - loss: 0.2847 - accuracy: 0.8917 - val_loss: 0.4407 - val_accuracy: 0.8554
Epoch 6/30
25000/25000 [==============================] - 4s 165us/sample - loss: 0.2227 - accuracy: 0.9184 - val_loss: 0.4764 - val_accuracy: 0.8570
Epoch 7/30
25000/25000 [===========================

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [3]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [5]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [6]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [7]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
vect = TfidfVectorizer()

In [0]:
X_train_dtm = vect.fit_transform(twenty_train.data)
X_test_dtm = vect.transform(twenty_test.data)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [15]:
logreg = LogisticRegression()
logreg.fit(X_train_dtm, twenty_train.target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Training Accuracy

In [0]:
y_pred_train = logreg.predict(X_train_dtm)

In [18]:
metrics.accuracy_score(twenty_train.target, y_pred_train)

0.9920248116969429

Testing Accuracy

In [0]:
y_pred_test = logreg.predict(X_test_dtm)

In [20]:
metrics.accuracy_score(twenty_test.target, y_pred_test)

0.8974700399467377